In [9]:
# ------------------------------
# - Envelope Strategy Backtest -
# ------------------------------

# --- IMPORT LIBS ---
import sys
sys.path.append('/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/live_tools/utilities')
sys.path.append('/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/backtest_tools/utilities')
#sys.path.append('/home/oddc/crypto_robot_live/my_code/live_tools/utilities'))
import ccxt
import ta
import pandas as pd
#from perp_bitget import PerpBitget
from data_manager import ExchangeDataManager
from backtesting import *
#from custom_indicators import CustomIndicators as ci
from perp_bybit import *
from custom_indicators import get_n_columns
from datetime import datetime
import time
import json
import pprint
import uuid
import logging

In [13]:
exchange_name = "binance"

intervals = ["1h"]

#coin_to_dl = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'XRP/USDT', 'DOGE/USDT', 'ADA/USDT', 'MATIC/USDT', 'DOT/USDT', 'SHIB/USDT', 'LTC/USDT', 'TRX/USDT', 'SOL/USDT', 'AVAX/USDT', 'LINK/USDT', 'ATOM/USDT', 'ETC/USDT', 'BCH/USDT', 'FIL/USDT', 'APE/USDT', 'VET/USDT', 'EGLD/USDT', 'EOS/USDT', 'SAND/USDT', 'CHZ/USDT', 'AAVE/USDT', 'AXS/USDT', 'MANA/USDT', 'CAKE/USDT', 'FTM/USDT', 'MKR/USDT', 'GRT/USDT']
coin_to_dl = ['BTC/USDT']

exchange = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download="/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/my_data//database/exchanges"
)

await exchange.download_data(
    coins=coin_to_dl, 
    intervals=intervals
)

	Récupération pour la paire BTC/USDT en timeframe 1h sur l'exchange binance...


100%|██████████| 57/57 [00:03<00:00, 15.94it/s]


In [14]:
exchange_name = "binance"

exchange = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download="/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/my_data//database/exchanges"
)

exchange.load_data(coin="BTC/USDT", interval="1h")

dfList = []
for pair in coin_to_dl:
    df = exchange.load_data(coin=pair, interval="1h")
    dfList.append(df)

print("Data load 100%")

Data load 100%


/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/backtest_tools/utilities/data_manager.py:114: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_name, index_col=0, parse_dates=True)
/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/backtest_tools/utilities/data_manager.py:114: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_name, index_col=0, parse_dates=True)


In [16]:
dfList

[                         open      high       low     close       volume
 date                                                                    
 2017-08-17 04:00:00   4261.48   4313.62   4261.32   4308.83    47.181009
 2017-08-17 05:00:00   4308.83   4328.69   4291.37   4315.32    23.234916
 2017-08-17 06:00:00   4330.29   4345.45   4309.37   4324.35     7.229691
 2017-08-17 07:00:00   4316.62   4349.99   4287.41   4349.99     4.443249
 2017-08-17 08:00:00   4333.32   4377.85   4333.32   4360.69     0.972807
 ...                       ...       ...       ...       ...          ...
 2023-06-26 16:00:00  30419.63  30432.00  29982.47  30100.47  4174.440250
 2023-06-26 17:00:00  30100.46  30170.38  29930.00  30084.63  3609.934460
 2023-06-26 18:00:00  30084.63  30239.62  30040.00  30213.36  1794.101770
 2023-06-26 19:00:00  30213.35  30306.96  30070.42  30243.97  1643.445440
 2023-06-26 20:00:00  30243.97  30290.00  30141.70  30161.78  1129.490120
 
 [51225 rows x 5 columns]]

In [31]:
# --- PARAMETERS & VARIABLES ---
# -- Account --
f = open('../live_tools/secret.json')
secret = json.load(f)
f.close()

exchange_name = 'bybit'
account_to_select = 'testnet_account'
production = True

# -- Coins & timeframe --
timeframe = '1h'
pair = "BTC/USDT:USDT"
leverage = 1
# logging.info(f"--- {pair} {timeframe} Leverage x {leverage} ---")

# -- Indicator variable --
ema_shifts = [0.01, 0.02, 0.03]
ema_period = 20

# -- Rules --
SL_ativation = False                # flag to set explicit stop loss
TP_activation = False               # flag to set explicit take profit
SL_percentage = 0.5                 # stop loss percentage applied to buy/sell price if flag activated
TP_percentage = 0.10                # not used
multi_ATR = 0.3                     # used to set take profit if flag activated
showLog = True                      #
max_coins_in_position = 1           # max number of coin positions
nLevel = len(ema_shifts)            # max number of open positions per coin
position_type = ["long", "short"]
open_position_asap = True
close_position_with_indicator = False

# -- Value initialisation of trade monitoring variables of each coin --
stopLoss = 0            # list of stop loss of each coin, set if SL_ativation is True
takeProfit = 5000000    # list of take profit if TP_activation is True
walletCoinArray = [0]     # list of number of coins longed / shorted
walletUsdArray = [0]      # list of USDT used to enter in the position of each coin
entryPrice = 0          # list of entry price of each coin
leverageEnter = 1       # list of leverage used for each coin
position = 'LONG'       # list of position directions for each coin
nPosition = 0           # list of number of positions in each coin
activePositions = 0     # number of open positions

# -- Formating of dataframes --
pd.set_option('display.max_columns', 5)             # Display any number of columns
pd.set_option('display.max_rows', 5)                # Display any number of rows
pd.set_option('display.expand_frame_repr', False)   # Don't wrap repr(DataFrame) across additional lines


In [21]:
# --- INITIALIZE EXCHANGE & GET BALANCE ---
# connect exchange
bybit = PerpBybit(
    apiKey=secret[account_to_select]["apiKey"],
    secret=secret[account_to_select]["secret"],
    default_type='swap',
    is_real=secret[account_to_select]["is_real"]
)

hey .. set sandbox mode ON


In [18]:
# --- FUNCTIONS ---
def open_long(row):
    if ('long' in position_type) and open_position_asap:
        return True
    else:
        return False

def close_long(row):
    if close_position_with_indicator:
        # some code
        return True
    else:
        return False

def open_short(row):
    if ('short' in position_type) and open_position_asap:
        return True
    else:
        return False

def close_short(row):
    if close_position_with_indicator:
        # some code
        return True
    else:
        return False


In [22]:
# --- POPULATE INDICATORS ---
sell_ema_values={}
buy_ema_values={}
df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)

df['ema_base'] = ta.trend.ema_indicator(close=df['close'], window=ema_period)
for i, shift in enumerate(ema_shifts, start=1):
    df[f'ema_high_{i}'] = df['ema_base'] * (1 + shift)
    df[f'ema_low_{i}'] = df['ema_base'] * (1 - shift)
    sell_ema_values[f'ema_high_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_high_{i}'])
    buy_ema_values[f'ema_low_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_low_{i}'])

In [23]:
df

,open,high,...,ema_high_3,ema_low_3
date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,...,NaN,NaN
2017-08-17 05:00:00,4308.83,4328.69,...,NaN,NaN
...,...,...,...,...,...
2023-06-26 19:00:00,30213.35,30306.96,...,31238.500091,29418.781639
2023-06-26 20:00:00,30243.97,30290.00,...,31222.131835,29403.366874


In [24]:
# -- Starting value --
startingBalance = 1000
makerFee = 0.0002*0.9
takerFee = 0.0002*0.9


In [25]:
# -- Definition of dfTrades, that will be the dataset to do your trades analyses --
dfTrades = None
dfTrades = pd.DataFrame(columns=['date', 'symbol','position', 'reason',
                                 'price', 'frais', 'fiat', 'coins', 'wallet'])

In [27]:
dfTestList = []
for df in dfList:
    dfTestList.append(df.loc['2023-01-01':])

In [29]:
walletCoinArray

0

In [32]:
for index, row in dfTestList[0].iterrows():
    # -- Check if you have one coin in your wallet --
    if (walletCoinArray.count(0) == len(walletCoinArray)) == False:
        # -- Iteration on all coin (upgrade possible: only check coin in your wallet) --
        for i in range(0,len(dfTestList)):
            leverageLong = 1
            leverageShort = 1
            # -- Check if you have more than 0 coin --
            if walletCoinArray[i] != 0:
                try:
                    actualRow = dfTestList[i].loc[index]
                    previousRow = dfTestList[i].loc[lastIndex]
                    prePreviousRow = dfTestList[i].loc[lastLastIndex]

                    # -- Take Profit Long --
                    if (actualRow['high'] > takeProfit[i] and position[i]=='LONG' and takeProfitActivation==True):
                        nPosition[i] = 0
                        sellPrice = takeProfit[i]
                        usd = usd + walletCoinArray[i] * entryPrice[i] / leverageEnter[i] + walletCoinArray[i] * (sellPrice - entryPrice[i])
                        fee = makerFee * walletCoinArray[i] * sellPrice
                        usd = usd - fee
                        # -- Set coin and equivalent usd to 0 after sold position --
                        walletCoinArray[i] = 0
                        walletUsdArray[i] = 0
                        activePositions -= 1

                        # -- LOG --
                        if showLog:
                            print("Sell COIN at Take Profit", sellPrice, '$ the', index)

                        # -- Add the trade to dfTrades to analyse it later --
                        myrow = {
                            'date': index,
                            'symbol': coin_to_dl[i],
                            'position': "Sell",
                            'reason': 'Sell Take Profit',
                            'price': sellPrice,
                            'frais': fee,
                            'fiat': usd,
                            'coins': 0,
                            'wallet': sum(walletUsdArray) + usd
                        }
                        dfTrades = dfTrades._append(myrow, ignore_index=True)

                    # -- Take Profit Short --
                    elif (actualRow['low'] < takeProfit[i] and position[i]=='SHORT' and takeProfitActivation==True):
                        nPosition[i] = 0
                        sellPrice = takeProfit[i]
                        usd = usd + walletCoinArray[i] * entryPrice[i] / leverageEnter[i] + walletCoinArray[i] * (entryPrice[i] - sellPrice)
                        fee = makerFee * walletCoinArray[i] * sellPrice
                        usd = usd - fee
                        # -- Set coin and equivalent usd to 0 after sold position --
                        walletCoinArray[i] = 0
                        walletUsdArray[i] = 0
                        activePositions -= 1

                        # -- LOG --
                        if showLog:
                            print("Sell COIN at Take Profit", sellPrice, '$ the', index)

                        # -- Add the trade to dfTrades to analyse it later --
                        myrow = {
                            'date': index,
                            'symbol': coin_to_dl[i],
                            'position': "Sell",
                            'reason': 'Sell Take Profit',
                            'price': sellPrice,
                            'frais': fee,
                            'fiat': usd,
                            'coins': 0,
                            'wallet': sum(walletUsdArray) + usd
                        }
                        dfTrades = dfTrades._append(myrow, ignore_index=True)

                    # -- Sell Market --
                    elif (closeLongCondition(actualRow,previousRow, prePreviousRow) != 0 and position[i] == 'LONG'):
                        nPosition[i] = 0
                        sellPrice = closeLongCondition(actualRow,previousRow, prePreviousRow)
                        usd = usd + walletCoinArray[i] * entryPrice[i] / leverageEnter[i] + walletCoinArray[i] * (sellPrice - entryPrice[i])
                        fee = makerFee * walletCoinArray[i] * sellPrice
                        usd = usd - fee
                        # -- Set coin and equivalent usd to 0 after sold position --
                        walletCoinArray[i] = 0
                        walletUsdArray[i] = 0
                        activePositions -= 1

                        # -- LOG --
                        if showLog:
                            print("Sell", coin_to_dl[i],"at", sellPrice, '$ the', index)

                        # -- Add the trade to DfTrades to analyse it later --
                        myrow = {
                            'date': index,
                            'symbol': coin_to_dl[i],
                            'position': "Sell",
                            'reason': 'Sell Market Order',
                            'price': sellPrice,
                            'frais': fee,
                            'fiat': usd,
                            'coins': 0,
                            'wallet': sum(walletUsdArray) + usd
                        }
                        dfTrades = dfTrades._append(myrow, ignore_index=True)

                    # -- Sell Market --
                    elif (closeShortCondition(actualRow,previousRow,prePreviousRow) != 0 and position[i] == 'SHORT'):
                        nPosition[i] = 0
                        sellPrice = closeShortCondition(actualRow,previousRow,prePreviousRow)
                        usd = usd + walletCoinArray[i] * entryPrice[i] / leverageEnter[i] + walletCoinArray[i] * (entryPrice[i] - sellPrice)
                        fee = makerFee * walletCoinArray[i] * sellPrice
                        usd = usd - fee
                        # -- Set coin and equivalent usd to 0 after sold position --
                        walletCoinArray[i] = 0
                        walletUsdArray[i] = 0
                        activePositions -= 1

                        # -- LOG --
                        if showLog:
                            print("Sell", coin_to_dl[i],"at", sellPrice, '$ the', index)

                        # -- Add the trade to DfTrades to analyse it later --
                        myrow = {
                            'date': index,
                            'symbol': coin_to_dl[i],
                            'position': "Sell",
                            'reason': 'Sell Market Order',
                            'price': sellPrice,
                            'frais': fee,
                            'fiat': usd,
                            'coins': 0,
                            'wallet': sum(walletUsdArray) + usd
                        }
                        dfTrades = dfTrades._append(myrow, ignore_index=True)


                except Exception as e:
                    error = f"{type(e).__name__} at line {e.__traceback__.tb_lineno}: {e}"
    # -- Buy market order --
    # -- Check if you can open a new position --
    if activePositions < maxPositions:
        # -- Check buy condition for each coin of list --
        for i in range(0,len(dfTestList)):
            try:
                actualRow = dfTestList[i].loc[index]
                previousRow = dfTestList[i].loc[lastIndex]
                leverageLong = 1
                leverageShort = 1
                # -- Buy condition --
                buyPrice = openLongCondition(actualRow, previousRow)
                if (buyPrice != 0 and ((activePositions < maxPositions and nPosition[i] == 0) 
                             or (nPosition[i] != 0 and nPosition[i] < nLevel))):
                    #buyPrice = actualRow['close']
                    # -- Define size of the position --
                    usdMultiplier = 1/nLevel * 1/(maxPositions-activePositions) * leverageLong
                    fee = takerFee * usd * usdMultiplier
                    usd = usd - fee
                    coin = (usd * usdMultiplier) / buyPrice
                    usd = usd - (usd * usdMultiplier / leverageLong)
                    leverageEnter[i] = leverageLong
                    entryPrice[i] = (buyPrice + entryPrice[i]*nPosition[i])/(nPosition[i]+1)
                    position[i] = 'LONG'
                    # -- Set coin and equivalent usd to size of position after open position --
                    walletCoinArray[i] += coin
                    walletUsdArray[i] += coin * actualRow['close'] / leverageLong
                    if nPosition[i] == 0:
                        activePositions += 1
                    nPosition[i] += 1
                    # -- Create a Stop Loss --
                    if StopLossActivation:
                        stopLoss[i] = buyPrice - SlPct * buyPrice
                    # -- Create a Take Profit --
                    if takeProfitActivation:
                        TpPct = actualRow['ATR']*multiATR
                        takeProfit[i] = buyPrice + TpPct
                    # -- LOG --
                    if showLog:
                        print("Buy", coin_to_dl[i],"at", buyPrice, '$ the', index)

                    # -- Add the trade to dfTrades to analyse it later --
                    myrow = {
                        'date': index,
                        'symbol': coin_to_dl[i],
                        'position': "Buy Long",
                        'reason': 'Buy Market Order',
                        'price': buyPrice,
                        'frais': fee,
                        'fiat': usd,
                        'coins': coin,
                        'wallet': sum(walletUsdArray) + usd
                    }
                    dfTrades = dfTrades._append(myrow, ignore_index=True)

                elif(openShortCondition(actualRow, previousRow) != 0 
                        and ((activePositions < maxPositions and nPosition[i] == 0) 
                             or (nPosition[i] != 0 and nPosition[i] < nLevel))):
                    buyPrice = openShortCondition(actualRow, previousRow)
                    # -- Define size of the position --
                    usdMultiplier = 1/nLevel * 1/(maxPositions-activePositions) * leverageShort
                    fee = takerFee * usd * usdMultiplier
                    usd = usd - fee
                    coin = (usd * usdMultiplier) / buyPrice
                    usd = usd - (usd * usdMultiplier / leverageShort)
                    leverageEnter[i] = leverageShort
                    entryPrice[i] = (buyPrice + entryPrice[i]*nPosition[i])/(nPosition[i]+1)
                    position[i] = 'SHORT'
                    # -- Set coin and equivalent usd to size of position after open position --
                    walletCoinArray[i] += coin
                    walletUsdArray[i] += coin * actualRow['close'] / leverageShort
                    if nPosition[i] == 0:
                        activePositions += 1
                    nPosition[i] += 1
                    # -- Create a Stop Loss --
                    if StopLossActivation:
                        stopLoss[i] = buyPrice + SlPct * buyPrice
                    # -- Create a Take Profit --
                    if takeProfitActivation:
                        TpPct = actualRow['ATR']*multiATR
                        takeProfit[i] = buyPrice - TpPct

                    # -- LOG --
                    if showLog:
                        print("Buy", coin_to_dl[i],"at", buyPrice, '$ the', index)

                    # -- Add the trade to dfTrades to analyse it later --
                    myrow = {
                        'date': index,
                        'symbol': coin_to_dl[i],
                        'position': "Buy Short",
                        'reason': 'Buy Market Order',
                        'price': buyPrice,
                        'frais': fee,
                        'fiat': usd,
                        'coins': coin,
                        'wallet': sum(walletUsdArray) + usd
                    }
                    dfTrades = dfTrades._append(myrow, ignore_index=True)
            except Exception as e:
                error = f"{type(e).__name__} at line {e.__traceback__.tb_lineno}: {e}"

    # -- Keep last index to define last row --  
    lastLastIndex = lastIndex          
    lastIndex = index

print("Final wallet", sum(walletUsdArray) + usd)

NameError: name 'maxPositions' is not defined